In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir = './images_hibiscus/train'
test_dir = './images_hibiscus/test'
validation_dir = './images_hibiscus/val'

# Data augmentation for training data
train_datagen = ImageDataGenerator(rescale=1./255)

# Only rescaling for validation and test data
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Load base models without the top layers
input_tensor = Input(shape=(224, 224, 3))
vgg19_base = VGG19(weights='imagenet', include_top=False, input_tensor=input_tensor)
densenet_base = DenseNet121(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze base model layers
for layer in vgg19_base.layers:
    layer.trainable = False
for layer in densenet_base.layers:
    layer.trainable = False

# Extract features
vgg19_output = GlobalAveragePooling2D()(vgg19_base.output)
densenet_output = GlobalAveragePooling2D()(densenet_base.output)

# Concatenate features
merged = Concatenate()([vgg19_output, densenet_output])
merged = Dense(1024, activation='relu')(merged)
predictions = Dense(1, activation='sigmoid')(merged)

# Define final model
model = Model(inputs=input_tensor, outputs=predictions)
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=15
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

Found 2092 images belonging to 2 classes.
Found 450 images belonging to 2 classes.
Found 450 images belonging to 2 classes.


C:\Users\kavit\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 1209s 18s/step - accuracy: 0.8784 - loss: 0.3526 - val_accuracy: 0.9978 - val_loss: 0.0598
Epoch 2/15
 1/65 ━━━━━━━━━━━━━━━━━━━━ 15:28 15s/step - accuracy: 1.0000 - loss: 0.0580

C:\Users\kavit\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


65/65 ━━━━━━━━━━━━━━━━━━━━ 218s 3s/step - accuracy: 1.0000 - loss: 0.0580 - val_accuracy: 0.9978 - val_loss: 0.0592
Epoch 3/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 1152s 18s/step - accuracy: 0.9863 - loss: 0.0636 - val_accuracy: 1.0000 - val_loss: 0.0301
Epoch 4/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 219s 3s/step - accuracy: 1.0000 - loss: 0.0581 - val_accuracy: 1.0000 - val_loss: 0.0297
Epoch 5/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 1163s 18s/step - accuracy: 0.9955 - loss: 0.0377 - val_accuracy: 1.0000 - val_loss: 0.0198
Epoch 6/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 218s 3s/step - accuracy: 1.0000 - loss: 0.0204 - val_accuracy: 1.0000 - val_loss: 0.0197
Epoch 7/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 1148s 18s/step - accuracy: 0.9979 - loss: 0.0233 - val_accuracy: 1.0000 - val_loss: 0.0148
Epoch 8/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 220s 3s/step - accuracy: 1.0000 - loss: 0.0099 - val_accuracy: 1.0000 - val_loss: 0.0147
Epoch 9/15
65/65 ━━━━━━━━━━━━━━━━━━━━ 1149s 18s/step - accuracy: 0.9983 - loss: 0.0176 - val_accuracy: 1.0000 - val_